# EPANET-Turbo Linux Build Verification (M5)

This notebook verifies the compilation and runtime of EPANET-Turbo on a Linux environment (Google Colab/Ubuntu).

**Objective**: Build `libepanet2.so` from **Private Source** (uploaded to Google Drive) since the public repo is binary-only.

In [ ]:
# 1. Install Prerequisites
!apt-get update
!apt-get install -y cmake build-essential libomp-dev

## 2. Load Source from Google Drive

Assumes you have uploaded the `epanet-turbo` folder to the root of your Google Drive.

In [ ]:
from google.colab import drive
import os
import shutil

# Mount Drive
drive.mount('/content/drive')

# Define paths
# CHANGE THIS if your folder is named differently or in a subfolder
drive_source_path = "/content/drive/MyDrive/epanet-turbo"
local_source_path = "/content/epanet-turbo"

# Copy source to local runtime (faster compilation)
if os.path.exists(local_source_path):
    shutil.rmtree(local_source_path)

print(f"Copying from {drive_source_path} to {local_source_path}...")
shutil.copytree(drive_source_path, local_source_path)
print("✅ Source loaded.")

# Fix Private Source Structure
# CMake expects 'src', but private repo might hold it as 'private_src'
if os.path.exists(os.path.join(local_source_path, 'private_src')):
    print("🔒 Detected 'private_src', renaming to 'src' for build...")
    if os.path.exists(os.path.join(local_source_path, 'src')):
        shutil.rmtree(os.path.join(local_source_path, 'src'))
    os.rename(os.path.join(local_source_path, 'private_src'), os.path.join(local_source_path, 'src'))

%cd {local_source_path}
!ls -la src/ | head -n 5

## 3. Build with CMake

In [ ]:
!mkdir -p build
%cd build
!cmake ..
!make -j4

## 4. Verify Binaries

In [ ]:
!ls -lh *.so

## 5. Python Integration Test

In [ ]:
%cd /content/epanet-turbo
%pip install polars numpy

import ctypes
import os

# Manually load the built shared libraries for testing
so_path_serial = "./build/libepanet2.so"
so_path_openmp = "./build/libepanet2_openmp.so"

def verify_so(path, name):
    if not os.path.exists(path):
        print(f"❌ {name} not found at {path}")
        return
    
    try:
        lib = ctypes.CDLL(path)
        # Check ENT_engine_id
        if hasattr(lib, "ENT_engine_id"):
             lib.ENT_engine_id.restype = ctypes.c_char_p
             eid = lib.ENT_engine_id().decode()
             print(f"✅ {name}: ENT_engine_id = {eid}")
        else:
             print(f"⚠️ {name}: ENT_engine_id not found (Old version?)")
             
        # Check ENT_set_node_values (Batch API)
        if hasattr(lib, "ENT_set_node_values"):
            print(f"✅ {name}: Batch API found")
        else:
            print(f"❌ {name}: Batch API missing")
            
    except Exception as e:
        print(f"❌ {name} load failed: {e}")

verify_so(so_path_serial, "Serial Engine")
verify_so(so_path_openmp, "OpenMP Engine")